# Control : MC Control

## Import the library

In [1]:
import random
import numpy as np

## Environment Class

In [2]:
class GridWorld():
    def __init__(self):
        self.x = 0
        self.y = 0

    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션: 아래쪽
        if a==0:
            self.move_left()
        elif a==1:
            self.move_up()
        elif a==2:
            self.move_right()
        elif a==3:
            self.move_down()

        reward = -1           # 보상은 항상 -1로 고정
        done = self.is_done() # episode 종료 여부
        return (self.x, self.y), reward, done

    def move_right(self):
        self.y += 1
        if self.y > 3:
            self.y = 3

    def move_left(self):
        if self.y==0:
            pass
        elif self.y==3 and self.x in [0,1,2]:
            pass
        elif self.y==5 and self.x in [2,3,4]:
            pass
        else:
            self.y -= 1

    def move_right(self):
        if self.y==1 and self.x in [0,1,2]:
            pass
        elif self.y==3 and self.x in [2,3,4]:
            pass
        elif self.y==6:
            pass
        else:
            self.y += 1

    def move_up(self):
        if self.x==0:
            pass
        elif self.x==3 and self.y==2:
            pass
        else:
            self.x -= 1

    def move_down(self):
        if self.x==4:
            pass
        elif self.x==1 and self.y==4:
            pass
        else:
            self.x+=1

    def is_done(self):
        if self.x==4 and self.y==6: # 목표 지점인 (4,6)에 도달하면 끝난다
            return True
        else:
            return False

    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

## QAgent Class

In [3]:
class QAgent():
    def __init__(self):
        # q벨류를 저장하는 변수. 모두 0으로 초기화
        # action 4가지에 따른 각 q 벨류를 모두 저장하기 위해
        # (5,7) table에 차원 (4)을 추가하여 a_0~3에 따른 q벨류를 모두 기록
        self.q_table = np.zeros((5, 7, 4))
        self.eps = 0.9
        self.alpha = 0.01
        self.gamma = 1

    def select_action(self, s):
        # eps-greedy로 액션을 선택
        # 이전 ch4에서 정책(𝝅) 동서남북 0.25로 고정했던 것과 비교
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = self.q_table[x,y,:]  # s에서 action에 따른 4가지 q값 불러오기
            action = np.argmax(action_val)    # 4개의 q값 중 가장 높은 것의 index 추출(action 번호)
        return action

    def update_table(self, history):
        # 한 에피소드에 해당하는 history를 입력으로 받아 q 테이블의 값을 업데이트 한다
        cum_reward = 0
        for transition in history[::-1]:
            s, a, r, s_prime = transition
            x,y = s
            # 몬테 카를로 방식을 이용하여 업데이트
            # q(s,a) = q(s,a) + α * (G - q(s,a))
            self.q_table[x,y,a] = self.q_table[x,y,a] + self.alpha * (cum_reward - self.q_table[x,y,a])
            # G_t = R_t+1 + γG_t+1
            cum_reward = r + self.gamma*cum_reward

    def anneal_eps(self):  # ε decay
        self.eps -= 0.03
        self.eps = max(self.eps, 0.1)

    def show_table(self):
        # 학습이 각 위치에서 어느 액션의 q 값이 가장 높았는지 보여주는 함수
        q_lst = self.q_table.tolist()
        data = np.zeros((5,7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

## main

In [4]:
def main():
    env = GridWorld()
    agent = QAgent()

    for n_epi in range(1000): # 총 1,000 에피소드 동안 학습
        done = False
        history = []

        s = env.reset()
        while not done: # 한 에피소드가 끝날 때 까지
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            history.append((s, a, r, s_prime))
            s = s_prime
        # 한 에피소드가 끝난 후 히스토리를 이용하여 에이전트를 업데이트
        agent.update_table(history)
        agent.anneal_eps()

    agent.show_table() # 학습이 끝난 결과를 출력

In [5]:
if __name__ == '__main__':
    main()

[[3. 3. 0. 2. 2. 2. 3.]
 [2. 3. 0. 2. 2. 3. 3.]
 [2. 3. 0. 1. 0. 2. 3.]
 [2. 2. 2. 1. 0. 3. 3.]
 [2. 3. 1. 0. 0. 2. 0.]]
